## Airbnb project

In [227]:
import pandas as pd
import geopandas as gpd
import geoplot
import geoplot.crs as gcrs
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString
import seaborn as sns
import numpy as np
import shapely
from geopy import distance

In [228]:
hood = gpd.read_file('neighbourhoods.geojson')
metro = gpd.read_file('metro.geojson')
df = pd.read_csv('listings.csv')

# Metro dataset

In [229]:
df2 = metro.copy()
for i in range(len(df2)):
    if not '/' in df2['LINHA'][i]:
        df2.drop([i], axis=0, inplace=True)
df2 = df2.reset_index()

In [230]:
for i in range(len(metro)):
    if '/' in  metro['LINHA'][i]:
        metro['LINHA'][i] = metro['LINHA'][i].split('/')[0]

for j in range(len(df2)):
    df2['LINHA'][j] = df2['LINHA'][j].split('/')[1]
    
metro = pd.concat([metro, df2])

/var/folders/10/b37vvtzs1hx0zykkgsrfypbr0000gn/T/ipykernel_32469/857219963.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metro['LINHA'][i] = metro['LINHA'][i].split('/')[0]
/var/folders/10/b37vvtzs1hx0zykkgsrfypbr0000gn/T/ipykernel_32469/857219963.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['LINHA'][j] = df2['LINHA'][j].split('/')[1]


In [231]:
metro = pd.concat([metro, df2])

In [232]:
colors = {'Verde': 'green', 
          'Azul': 'blue', 
          'Azul/Verde': 'blue',
          'Amarela': 'red', 
          'Azul/Amarela': 'red',
          'Azul/Vermelha': 'blue',
          'Amarela/Vermelha': 'red',
          'Verde/Vermelha': 'green',
          'Amarela/Verde': 'red',
          'Vermelha': 'violet'
         }

In [233]:
metro['color_eng'] = (metro['LINHA']).map(colors)

In [234]:
yellow = metro[metro.LINHA == 'Amarela'].reset_index()
blue = metro[metro.LINHA == 'Azul'].reset_index()
green = metro[metro.LINHA == 'Verde'].reset_index()
red = metro[metro.LINHA == 'Vermelha'].reset_index()

In [235]:
#ax = geoplot.polyplot(hood, edgecolor='white', facecolor='lightgray', figsize=(12, 15))
#geoplot.pointplot(metro, ax=ax, hue='LINHA')
#gpd.GeoSeries([LineString(blue.geometry)]).plot(ax=ax)

# Metro Network

In [236]:
import networkx as nx
from pyvis.network import Network

In [237]:
metro_graph = pd.read_excel('metro.xlsx')

In [238]:
metro_graph = metro_graph.dropna()
metro_graph = metro_graph.drop(columns = ['NOME', 'LINHA', 'Unnamed: 0', 'OBJECTID'])

In [239]:
metro_graph.head()

,Origin,Destination
0,Reboleira,Amadora Este
1,Amadora Este,Alfornelos
2,Alfornelos,Pontinha
3,Pontinha,Carnide
4,Carnide,Colégio Militar


## Set up the graph

In [240]:
G = nx.Graph()
G = nx.from_pandas_edgelist(metro_graph, 'Origin', 'Destination')

In [241]:
metro_dict = dict(zip(metro.NOME, metro.geometry))

In [242]:
metro_graph['p1'] = (metro_graph['Origin']).map(metro_dict)
metro_graph['p2'] = (metro_graph['Destination']).map(metro_dict)

In [243]:
a = []
for i in range(len(metro_graph)):
    a.append((LineString([metro_graph.p1[i],metro_graph.p2[i]])))

metro_graph['LineString'] = a

In [244]:
metro_graph = gpd.GeoDataFrame(metro_graph)

## Calculate distance between nodes

In [245]:
from shapely.geometry import LineString
from shapely.ops import transform
from functools import partial
import pyproj

In [246]:
def transform_linestring(df):
    
    lenghts = []
    project = partial(pyproj.transform, pyproj.Proj('EPSG:4326'), pyproj.Proj('EPSG:32633'))
    for line in df.LineString:
        lenghts.append(int(transform(project, line).length))
    
    df['distance_meters'] = lenghts

In [247]:
transform_linestring(metro_graph)

In [248]:
G_distance = nx.from_pandas_edgelist(metro_graph, 'Origin', 'Destination',['distance_meters'])

### Sort based on degree

In [249]:
#degree_sort = sorted(G_distance.degree(), key=lambda x: x[1], reverse=True)
degrees = {node:val for (node, val) in G_distance.degree()}

### Degrees

In [250]:
closeness = nx.closeness_centrality(G_distance, distance='distance_meters')
#print(max(closeness, key=closeness.get))

In [251]:
between = nx.betweenness_centrality(G_distance, weight='distance_meters')
#print(max(between, key=between.get))

In [252]:
eigen = nx.eigenvector_centrality(G_distance,max_iter=10000, weight='distance_meters')

In [253]:
centrality = nx.degree_centrality(G_distance)

In [254]:
betweness = pd.DataFrame.from_dict(between, orient='index').rename(columns = {0: 'Betweness'}).reset_index().rename(columns = {'index': 'station'})
closeness = pd.DataFrame.from_dict(closeness, orient='index').rename(columns = {0: 'Closeness'}).reset_index().rename(columns = {'index': 'station'})
eigeness = pd.DataFrame.from_dict(eigen, orient='index').rename(columns = {0: 'Eigeness'}).reset_index().rename(columns = {'index': 'station'})
central = pd.DataFrame.from_dict(centrality, orient='index').rename(columns = {0: 'Centrality'}).reset_index().rename(columns = {'index': 'station'})
Degree = pd.DataFrame.from_dict(degrees, orient='index').rename(columns = {0: 'Degree'}).reset_index().rename(columns = {'index': 'station'})
weight = pd.merge(betweness, closeness, on='station')
weight = pd.merge(weight, eigeness, on='station')
weight = pd.merge(weight, central, on='station')
weight = pd.merge(weight, Degree, on='station')

In [255]:
weight['mean'] = weight.iloc[:,1:4].mean(axis=1)

In [256]:
weight.sort_values('mean').iloc[::-1].head()

,station,Betweness,Closeness,Eigeness,Centrality,Degree,mean
23,Campo Grande,0.227041,0.000147,0.626953,0.081633,4,0.284714
34,Alameda,0.458333,0.000209,0.135006,0.081633,4,0.197849
31,Alvalade,0.102891,0.000157,0.471594,0.040816,2,0.191548
27,Saldanha,0.443027,0.000215,0.121756,0.081633,4,0.188333
24,Cidade Universitária,0.143707,0.000155,0.353409,0.040816,2,0.165757


In [257]:
metro.head()

,OBJECTID,NOME,COD_SIG,IDTIPO,SITUACAO,LINHA,GlobalID,geometry,index,color_eng
0,1,Cais do Sodré,35,9008,Linha existente,Verde,d5542849-632f-4127-a009-3988731f3509,POINT (-9.14609 38.70627),NaN,green
1,2,Terreiro do Paço,34,9008,Linha existente,Azul,9a90a59a-8b21-4fd7-b9f2-3a633c4cdd07,POINT (-9.13419 38.70728),NaN,blue
2,3,Baixa Chiado,33,9008,Linha existente,Azul,18279178-37b5-4bf9-9136-c67aa97d6559,POINT (-9.14015 38.71057),NaN,blue
3,4,Santa Apolónia,37,9008,Linha existente,Azul,ebb35126-e7bf-4c7f-b3a4-deb2f3b899a0,POINT (-9.12241 38.71404),NaN,blue
4,5,Rossio,43,9008,Linha existente,Verde,302467d8-12a1-4052-86ac-8a33be669894,POINT (-9.13792 38.71418),NaN,green


In [258]:
lines_dict = dict(zip(metro.NOME, metro.LINHA))
station_coord = dict(zip(metro.NOME, metro.geometry))

In [259]:
weight['line_color'] = (weight['station']).map(lines_dict)
weight['Coordinate'] = (weight['station']).map(station_coord)

## Function that gives the distance in meters from a specified point to each metro station

The idea is to have an interactive weight attribution where we are the ones choosing the point of interest, giving coordinates which can be found on this website: https://www.latlong.net/place/rossio-square-lisbon-portugal-23131.html.
Then we assign a score to that monument and depending on how near the metro station is the more importance it gets.

In [260]:
def distance_grade(longlat_tuple, grade, df, name_of_place):
    
    col_name = 'Distance from ' + str(name_of_place)
    
    from_place_list = [] #list of distances
    location = Point(longlat_tuple) #get location of the monument
    project = partial(pyproj.transform, pyproj.Proj('EPSG:4326'), pyproj.Proj('EPSG:32633'))
    for i in df.Coordinate: #iterate over the coordinates of each metro station
        line_string = shapely.geometry.LineString([i,location]) #construct a linestring between the station and the monument
        trans = transform(project, line_string) #transform the linestring in meters
        #from_place_list.append(int(trans.length))
        
    #further implementation to assign weight (importance) to each monument
        if int(trans.length) < 2000: #if the distance is less than 4 km from the monument
            from_place_list.append(grade) 
        elif int(trans.length) < 4000:
            from_place_list.append(grade*0.5)
        elif int(trans.length) < 8000:
            from_place_list.append(grade*0.25)
        else:
            from_place_list.append(0)
            
    df[col_name] = from_place_list

## Implement for a list of monuments

In [261]:
list_of_monuments = {'Mosteiro dos Jerónimos': ((-9.204487, 38.697819), 5.5),
                    'Oceanário de Lisboa': ((-9.0937, 38.7635), 6),
                    'Museu Nacional de Arte Antiga': ((-9.141161, 38.708460), 4),
                    'Torre de Belem': ((-9.2147, 38.6967), 8),
                    'Elevador de Santa Justa': ((-9.139221, 38.712173), 9),
                    'Padrão dos Descobrimentos': ((-9.205712, 38.693596), 6),
                    'Praça do Comércio': ((-9.136744, 38.707779), 10),
                    'Museu de Arte, Arquitectura e Tecnologia': ((-9.194453, 38.695927), 6),
                    }

In [262]:
for i in list_of_monuments:
    distance_grade(list_of_monuments[i][0], list_of_monuments[i][1], weight, i)

In [263]:
weight['Score'] = weight.iloc[:,9:].mean(axis=1).round(2)
weight = weight.drop(columns = weight.iloc[:,9:-1].columns.tolist())
weight = weight.drop(columns = weight.iloc[:,1:4].columns.tolist())